In [1]:
#读取csv格式文件
import pandas as pd
path = './' #用户的路径环境
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import lightgbm as lgb
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')


In [2]:
test['y']=-1
cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']
feature1 = cate_features + num_features
data = pd.concat([train,test])

In [3]:
def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name
ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)

In [4]:
def feat_count(df, df_feature,fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % ("_".join(fe[0]))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left")#.fillna(0)
    return df,df_count.columns[1]


In [5]:
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']
crossf=[]
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
    crossf+=['count_'+str(d)+'_'+str(s),'nunique_'+str(d)+'_'+str(s)]
    temp=data.groupby(d)[s].agg({'count_'+str(d)+'_'+str(s):'count','nunique_'+str(d)+'_'+str(s):'nunique'}).reset_index()
    data=pd.merge(data,temp,on=d,how='left')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if __name__ == '__main__':
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.75it/s]


In [6]:
for d in tqdm(sparse_feature):
    aggs={}
    for s in dense_feature:
        aggs[s]=['count','nunique']
    crossf+=[str(d)+'_mean_'+str(s),str(d)+'_max_'+str(s),str(d)+'_min_'+str(s),str(d)+'_std_'+str(s)]
    temp = data.groupby(d)[s].agg({str(d)+'_mean_'+str(s):'mean',str(d)+'_max_'+str(s):'max',str(d)+'_min_'+str(s):'min',str(d)+'_std_'+str(s):'std'}).reset_index()
    data=pd.merge(data,temp,on=d,how='left')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.94it/s]


In [7]:
#ID这里好像没用'ID',
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.model_selection import KFold

num_features=ll+crossf+num_features


one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']


data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))





for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
        
vector_feature = ['new_con']


train=data[data.y!=-1]

# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
test=data[data.y==-1]
res=test[['ID']]
# test=test.drop(['ID'],axis=1)
train_x=train[num_features]
test_x=test[num_features]
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')


cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

# trainxx, vali_x, trainyy, vali_y = train_test_split(train_x,train_y,test_size=0.2, random_state=2018)
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_y=train['y']
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5
res.to_csv('./basesubmission.csv', index=False)






D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before

[1]	training's auc: 0.90948	valid_1's auc: 0.892429
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.914788	valid_1's auc: 0.896187
[3]	training's auc: 0.931149	valid_1's auc: 0.919877
[4]	training's auc: 0.928887	valid_1's auc: 0.920643
[5]	training's auc: 0.925952	valid_1's auc: 0.9181
[6]	training's auc: 0.922877	valid_1's auc: 0.917704
[7]	training's auc: 0.920077	valid_1's auc: 0.915426
[8]	training's auc: 0.927163	valid_1's auc: 0.92065
[9]	training's auc: 0.925412	valid_1's auc: 0.918334
[10]	training's auc: 0.923238	valid_1's auc: 0.916396
[11]	training's auc: 0.928929	valid_1's auc: 0.920761
[12]	training's auc: 0.927523	valid_1's auc: 0.920754
[13]	training's auc: 0.925933	valid_1's auc: 0.91955
[14]	training's auc: 0.929679	valid_1's auc: 0.9223
[15]	training's auc: 0.933009	valid_1's auc: 0.924856
[16]	training's auc: 0.931927	valid_1's auc: 0.92377
[17]	training's auc: 0.934794	valid_1's auc: 0.926917
[18]	training's auc: 0.936627	valid_

[151]	training's auc: 0.959197	valid_1's auc: 0.938862
[152]	training's auc: 0.959283	valid_1's auc: 0.938843
[153]	training's auc: 0.959357	valid_1's auc: 0.938814
[154]	training's auc: 0.959463	valid_1's auc: 0.938804
[155]	training's auc: 0.959564	valid_1's auc: 0.938785
[156]	training's auc: 0.959648	valid_1's auc: 0.938854
[157]	training's auc: 0.959738	valid_1's auc: 0.938811
[158]	training's auc: 0.959834	valid_1's auc: 0.938815
[159]	training's auc: 0.95992	valid_1's auc: 0.938832
[160]	training's auc: 0.960016	valid_1's auc: 0.938842
[161]	training's auc: 0.960093	valid_1's auc: 0.93891
[162]	training's auc: 0.960171	valid_1's auc: 0.938922
[163]	training's auc: 0.960297	valid_1's auc: 0.939071
[164]	training's auc: 0.960401	valid_1's auc: 0.939131
[165]	training's auc: 0.960488	valid_1's auc: 0.93915
[166]	training's auc: 0.960592	valid_1's auc: 0.939105
[167]	training's auc: 0.960659	valid_1's auc: 0.93905
[168]	training's auc: 0.960738	valid_1's auc: 0.939052
[169]	training

[300]	training's auc: 0.971172	valid_1's auc: 0.939686
[301]	training's auc: 0.971237	valid_1's auc: 0.939713
[302]	training's auc: 0.971303	valid_1's auc: 0.939676
[303]	training's auc: 0.971365	valid_1's auc: 0.93971
[304]	training's auc: 0.971455	valid_1's auc: 0.939726
[305]	training's auc: 0.971517	valid_1's auc: 0.939721
[306]	training's auc: 0.971589	valid_1's auc: 0.939717
[307]	training's auc: 0.97166	valid_1's auc: 0.939699
[308]	training's auc: 0.971725	valid_1's auc: 0.939716
[309]	training's auc: 0.971794	valid_1's auc: 0.939714
[310]	training's auc: 0.971866	valid_1's auc: 0.939711
[311]	training's auc: 0.971925	valid_1's auc: 0.939706
[312]	training's auc: 0.971986	valid_1's auc: 0.939706
[313]	training's auc: 0.972026	valid_1's auc: 0.939746
[314]	training's auc: 0.972095	valid_1's auc: 0.939742
[315]	training's auc: 0.972161	valid_1's auc: 0.939751
[316]	training's auc: 0.972228	valid_1's auc: 0.939704
[317]	training's auc: 0.972294	valid_1's auc: 0.939703
[318]	traini

[449]	training's auc: 0.979824	valid_1's auc: 0.940012
[450]	training's auc: 0.979878	valid_1's auc: 0.940025
[451]	training's auc: 0.979939	valid_1's auc: 0.94005
[452]	training's auc: 0.979986	valid_1's auc: 0.940057
[453]	training's auc: 0.980035	valid_1's auc: 0.940046
[454]	training's auc: 0.980086	valid_1's auc: 0.940034
[455]	training's auc: 0.980139	valid_1's auc: 0.940038
[456]	training's auc: 0.980188	valid_1's auc: 0.940027
[457]	training's auc: 0.980218	valid_1's auc: 0.940006
[458]	training's auc: 0.980251	valid_1's auc: 0.939992
[459]	training's auc: 0.980299	valid_1's auc: 0.939983
[460]	training's auc: 0.980364	valid_1's auc: 0.939984
[461]	training's auc: 0.980415	valid_1's auc: 0.939993
[462]	training's auc: 0.980452	valid_1's auc: 0.93998
[463]	training's auc: 0.980483	valid_1's auc: 0.939959
[464]	training's auc: 0.980537	valid_1's auc: 0.939983
[465]	training's auc: 0.980588	valid_1's auc: 0.93998
[466]	training's auc: 0.980652	valid_1's auc: 0.939971
[467]	trainin

...

[118]	training's auc: 0.957241	valid_1's auc: 0.933527
[119]	training's auc: 0.957415	valid_1's auc: 0.933534
[120]	training's auc: 0.957511	valid_1's auc: 0.933551
[121]	training's auc: 0.95767	valid_1's auc: 0.933582
[122]	training's auc: 0.957783	valid_1's auc: 0.933657
[123]	training's auc: 0.957877	valid_1's auc: 0.933674
[124]	training's auc: 0.957988	valid_1's auc: 0.933713
[125]	training's auc: 0.95809	valid_1's auc: 0.933743
[126]	training's auc: 0.958244	valid_1's auc: 0.933746
[127]	training's auc: 0.958365	valid_1's auc: 0.933764
[128]	training's auc: 0.958466	valid_1's auc: 0.933764
[129]	training's auc: 0.958573	valid_1's auc: 0.933761
[130]	training's auc: 0.958667	valid_1's auc: 0.933806
[131]	training's auc: 0.958784	valid_1's auc: 0.933774
[132]	training's auc: 0.958867	valid_1's auc: 0.933795
[133]	training's auc: 0.958954	valid_1's auc: 0.933781
[134]	training's auc: 0.959043	valid_1's auc: 0.933837
[135]	training's auc: 0.959137	valid_1's auc: 0.933859
[136]	traini

[267]	training's auc: 0.970661	valid_1's auc: 0.934637
[268]	training's auc: 0.970738	valid_1's auc: 0.934627
[269]	training's auc: 0.970807	valid_1's auc: 0.934642
[270]	training's auc: 0.970873	valid_1's auc: 0.934649
[271]	training's auc: 0.970953	valid_1's auc: 0.93463
[272]	training's auc: 0.971014	valid_1's auc: 0.934657
[273]	training's auc: 0.971103	valid_1's auc: 0.934653
[274]	training's auc: 0.971174	valid_1's auc: 0.934693
[275]	training's auc: 0.97123	valid_1's auc: 0.934705
[276]	training's auc: 0.971309	valid_1's auc: 0.934698
[277]	training's auc: 0.971401	valid_1's auc: 0.934727
[278]	training's auc: 0.971468	valid_1's auc: 0.934752
[279]	training's auc: 0.971549	valid_1's auc: 0.934711
[280]	training's auc: 0.97163	valid_1's auc: 0.934695
[281]	training's auc: 0.971696	valid_1's auc: 0.934687
[282]	training's auc: 0.971749	valid_1's auc: 0.93468
[283]	training's auc: 0.971803	valid_1's auc: 0.934707
[284]	training's auc: 0.971876	valid_1's auc: 0.934705
[285]	training

[416]	training's auc: 0.979616	valid_1's auc: 0.93485
[417]	training's auc: 0.979656	valid_1's auc: 0.934865
[418]	training's auc: 0.979701	valid_1's auc: 0.934869
[419]	training's auc: 0.979746	valid_1's auc: 0.93483
[420]	training's auc: 0.979787	valid_1's auc: 0.934838
[421]	training's auc: 0.979838	valid_1's auc: 0.934822
[422]	training's auc: 0.979884	valid_1's auc: 0.934813
[423]	training's auc: 0.979933	valid_1's auc: 0.934807
[424]	training's auc: 0.979979	valid_1's auc: 0.93485
[425]	training's auc: 0.980028	valid_1's auc: 0.934843
[426]	training's auc: 0.980063	valid_1's auc: 0.93485
[427]	training's auc: 0.980102	valid_1's auc: 0.934867
[428]	training's auc: 0.980142	valid_1's auc: 0.934869
[429]	training's auc: 0.980193	valid_1's auc: 0.934872
[430]	training's auc: 0.980245	valid_1's auc: 0.934874
[431]	training's auc: 0.980286	valid_1's auc: 0.934877
[432]	training's auc: 0.980337	valid_1's auc: 0.93486
[433]	training's auc: 0.980383	valid_1's auc: 0.93483
[434]	training's

[565]	training's auc: 0.985767	valid_1's auc: 0.934712
[566]	training's auc: 0.985806	valid_1's auc: 0.934718
[567]	training's auc: 0.985834	valid_1's auc: 0.934703
[568]	training's auc: 0.985869	valid_1's auc: 0.934677
[569]	training's auc: 0.985907	valid_1's auc: 0.934708
[570]	training's auc: 0.985946	valid_1's auc: 0.934722
[571]	training's auc: 0.985978	valid_1's auc: 0.934723
[572]	training's auc: 0.986021	valid_1's auc: 0.934696
[573]	training's auc: 0.986051	valid_1's auc: 0.934715
[574]	training's auc: 0.986093	valid_1's auc: 0.934698
[575]	training's auc: 0.986158	valid_1's auc: 0.934727
[576]	training's auc: 0.986186	valid_1's auc: 0.934719
[577]	training's auc: 0.986218	valid_1's auc: 0.934739
[578]	training's auc: 0.986249	valid_1's auc: 0.934756
[579]	training's auc: 0.986275	valid_1's auc: 0.934753
[580]	training's auc: 0.9863	valid_1's auc: 0.934734
[581]	training's auc: 0.986329	valid_1's auc: 0.934739
[582]	training's auc: 0.986361	valid_1's auc: 0.934729
[583]	traini

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
